In [14]:
from pymongo import MongoClient
import pandas as pd
import re
import folium

In [15]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

db, coll = connectCollection('companies','companies')
#offices = {"$unwind": "$offices"},
#db.inventory.aggregate( [ { $unwind : "$sizes" } ] )
    


In [16]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

#unpack = [ { '$unwind' : '$offices' },{ '$project': {"_id":0} }, {"$out":"companies"}]
#companies.companies.aggregate(unpack)

db, coll = connectCollection('companies','companies')
interesting_companies = list(coll.find({"$unwind": "$offices"},
                                        {"$match": {"$and":
                                                    [{"founded_year":{"$gte":2009}},
                                                    {"deadpooled_year":None},
                                                    {"total_money_raised":{"$exists": True,"$ne":"$0"}},
                                                    {"offices.latitude": {"$exists": True, "$ne": None}},
                                                    {"offices.longitude": {"$exists": True, "$ne": None}}]}},                                     
                                      {"name":1,"founded_year": 1, "offices.latitude":1,
                                       "offices.longitude":1, "category_code":1, "location":1,
                                       "total_money_raised":1}))


#diferente = [{"$unwind":"offices"}]
#interesting_companies.companies.aggregate(diferente)


print(len(interesting_companies))

interesting_companies

TypeError: skip must be an instance of int

In [13]:
{ "$unwind": "$offices"},
    {"$match":{ "$and": [  {"founded_year":{"$gt":2000}},{"funding_rounds.raised_amount":{"$gt":1000000}}] }}

IndentationError: unexpected indent (<ipython-input-13-0ec6043b0312>, line 2)

In [ ]:
print(len(interesting_companies[2]['location']))

In [ ]:
companies = pd.DataFrame(interesting_companies)
print(len(companies))


In [ ]:
unpack = [ { '$unwind' : '$offices' },{ '$project': {"_id":0} }, {"$out":"companies"}]
db.coll.aggregate(unpack)

In [ ]:
db, coll = connectCollection('companies','companies')
interesting_companies = list(coll.find({"$and":
                                        [{"founded_year":{"$gte":2009}},
                                        {"deadpooled_year":None},
                                        {"total_money_raised":{"$exists": True,"$ne":"$0"}},
                                        {"offices.latitude": {"$exists": True, "$ne": None}},
                                        {"offices.longitude": {"$exists": True, "$ne": None}},
                                        {"$or":
                                        [{"category_code":"web"},
                                        {"category_code":"mobile"},
                                        {"category_code":"finance"},
                                        {"category_code":"games_video"},
                                        {"category_code":"analytics"},
                                        {"category_code": "software"}]}]},                                     
                                      {"name":1,"founded_year": 1, "offices.latitude":1,
                                       "offices.longitude":1, "category_code":1, "location":1,
                                       "total_money_raised":1}))

interesting_companies

In [ ]:
companies = companies.drop(['_id'], axis = 1)
companies.head()


In [ ]:
companies.total_money_raised = companies.total_money_raised.str.replace(r'(\d+)([A-Za-z])', r'\1 \2')
companies.total_money_raised = companies.total_money_raised.str.replace(r'(.)(\d+)', r'\1 \2')
companies.head()


In [ ]:
#companies3 = companies2[companies2["total_money_raised"].str.contains("M", na=True)]
#companies3
#df[df['model'].str.contains('ac')]

companies1 = companies[['M' in x for x in companies['total_money_raised']]]
companies1.head()

In [ ]:
companies1.to_json('updatedDB.json')

In [ ]:
print(companies1.to_json())

In [ ]:
map_city=folium.Map(location=[-111.9035,33.8171], zoom_start=12)
for company in companies1:
    folium.Marker(company[0]['location'][0][1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    
